In [1]:
!pip install flask-ngrok

In [ ]:
import sqlite3
import datetime
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when app is run

def get_db_connection():
    conn = sqlite3.connect('employee_data.db')
    conn.row_factory = sqlite3.Row
    return conn

@app.route("/", methods=["GET", "POST"])
def index():
    response = {}
    if request.method == "POST":
        query = request.json.get("query", "").lower()
        try:
            conn = get_db_connection()
            cursor = conn.cursor()

            if "show me all employees in the" in query:
                department = query.split("show me all employees in the")[1].strip().replace("department", "").strip()
                cursor.execute("SELECT * FROM Employees WHERE Department = ?", (department,))
                rows = cursor.fetchall()
                response["results"] = [dict(row) for row in rows]

            elif "who is the manager of the" in query:
                department = query.split("who is the manager of the")[1].strip().replace("department", "").strip()
                cursor.execute("SELECT Manager FROM Departments WHERE Name = ?", (department,))
                row = cursor.fetchone()
                response["result"] = row["Manager"] if row else "No manager found."

            elif "list all employees hired after" in query:
                date_str = query.split("list all employees hired after")[1].strip()
                try:
                    date = datetime.datetime.strptime(date_str, "%Y-%m-%d").date()
                    cursor.execute("SELECT * FROM Employees WHERE Hire_Date > ?", (date_str,))
                    rows = cursor.fetchall()
                    response["results"] = [dict(row) for row in rows]
                except ValueError:
                    response["error"] = "Invalid date format. Please use YYYY-MM-DD."

            elif "what is the total salary expense for the" in query:
                department = query.split("what is the total salary expense for the")[1].strip().replace("department", "").strip()
                cursor.execute("SELECT SUM(Salary) FROM Employees WHERE Department = ?", (department,))
                row = cursor.fetchone()
                response["result"] = row[0] if row else 0

            else:
                response["error"] = "I don't understand your query. Please try again."

            conn.close()

        except sqlite3.Error as e:
            response["error"] = f"Database error: {e}"
        except Exception as e:
            response["error"] = f"An error occurred: {e}"

    return jsonify(response)

if __name__ == "__main__":
    app.run()

In [ ]:
!pip install requests

In [ ]:
import requests

url = "<YOUR_NGROK_URL>"  # Replace with the ngrok URL

query = "Show me all employees in the Sales department."
response = requests.post(url, json={"query": query})
print(response.json())